This notebook contains examples for syncing memberships for different use cases. Before syncing memberships, ensure that all organisations have been imported.

In [ ]:
import requests
import gSheet_utils
import searchCLI
import membershipsUpdate_langs
import personsUpdate_langs

In [ ]:
base_url = "http://api.popit.sinarproject.org"
token = open('../oAuth/token_ms.txt').read()
headers = {'Authorization': token.rstrip()}
sub_langs = ['ms']

## Importing Company memberships

In the case of importing a govenment owned or linked private company, it is common that we might have a case where we'd split a company into child organisations as such:

- CompanyX (the parent organisation)
And its child organisations:
	- CompanyX_BOD (the Board of directors for the company)
	- CompanyX_management

When filling in the memberships template, we'd have to specify the parent-child relationships as in the memberships_Company_template in the templates folder.

In [ ]:
sheetID = 'gSheetID'

In our example, we'll have a spreadsheet containing 3 sheets following the naming format of memberships_orgName-suborgName:
- memberships_Company_X
- memberships_Company_X-Board_of_Directors
- memberships_Company_X-management

We can get a list of all the sheet names in our spreadsheet as such:

In [ ]:
#Get a list of all sheet names in the spreadsheet
sheetNames = gSheet_utils.getSheetNames(spreadsheetID)

The first memberships_CompanyX sheet can be synced as usual with:

In [ ]:
sheetName = 'memberships_Company_X'

df, col_AI_map = gSheet_utils.importGSheetAsDF(sheetID, sheetName)
gSheet_details = {'sheetID':sheetID, 'sheetName': sheetName, 'col_AI_map': col_AI_map, 'sub_langs':sub_langs}

#Get orgID of company by searching for it in Popit
orgID = searchCLI.searchCLI(base_url, orgName, 'organizations', 'name', 'othernames')    
#Import memberships under the orgID obtained
df.apply(lambda row: membershipsUpdate_langs.genPayload(base_url, headers, row, orgID, gSheet_details, sub_langs), axis=1)

However, for each of the suborganisations of Company X, we will have to import the memberships once under the suborganisation, and a second time under the parent organisation. For example, for the BOD suborganisation:

In [3]:
sheetName = 'memberships_Company_X-Board_of_Directors'

#Import gSheet as df
df, col_AI_map = gSheet_utils.importGSheetAsDF(sheetID, sheetName)
gSheet_details = {'sheetID':sheetID, 'sheetName': sheetName, 'col_AI_map': col_AI_map, 'sub_langs':sub_langs}

#Get name of org and parent org
orgName = ' '.join(sheetName.split('memberships_')[1].split('_'))
parentOrg = orgName.split('-')[0]

#Import under child org
orgID = searchCLI.searchCLI(base_url, orgName, 'organizations', 'name', 'othernames')    
df.apply(lambda row: membershipsUpdate_langs.genPayload(base_url, headers, row, orgID, gSheet_details, sub_langs), axis=1)

#Import under parent org
orgID = searchCLI.searchCLI(base_url, parentOrg, 'organizations', 'name', 'othernames')    
df.apply(lambda row: membershipsUpdate_langs.genPayload(base_url, headers, row, orgID, gSheet_details, sub_langs), axis=1)

We repeat this for all suborganisations in the spreadsheet.

We can simplify this process as a loop over the entire spreadsheet, which will import once if it's the base org, and twice if it's a sub organisation.

In [35]:
#Get a list of all sheet names in the spreadsheet
sheetNames = gSheet_utils.getSheetNames(spreadsheetID)

for sheetName in sheetNames:
    #Get name of org and parent org
    orgName = ' '.join(sheetName.split('memberships_')[1].split('_'))
    parentOrg = orgName.split('-')[0]
    
    #Import GSheet as df
    df, col_AI_map = gSheet_utils.importGSheetAsDF(sheetID, sheetName)
    gSheet_details = {'sheetID':sheetID, 'sheetName': sheetName, 'col_AI_map': col_AI_map, 'sub_langs':sub_langs}
    
    #If it's a sub org, import once under the parent org
    if parentOrg!=orgName:     
        orgID = searchCLI.searchCLI(base_url, parentOrg, 'organizations', 'name', 'othernames', []) #Find orgID from Popit
        print("Importing for {}".format(parentOrg)            
        df.apply(lambda row: membershipsUpdate_langs.genPayload(base_url, headers, row, orgID, gSheet_details, sub_langs), axis=1)

    #Import under the sub org
    orgID = searchCLI.searchCLI(base_url, orgName, 'organizations', 'name', 'othernames', [])
    print("Importing for {}".format(orgName))            
    df.apply(lambda row: membershipsUpdate_langs.genPayload(base_url, headers, row, orgID, gSheet_details, sub_langs), axis=1)






## Importing Government Department Org memberships

Government Department Orgs follow the pattern of having a parent department, and multiple sub-departments which vary from agency to agency.
When filling in the memberships template, we'd have to specify the parent-child relationships as in the memberships_GovDepartment_template in the templates folder.

As we did for the Companies case, we would have to loop over the entire spreadsheet, importing memberships once if it's the base org, and twice if it's a sub organisation.

In [ ]:
#Get a list of all sheet names in the spreadsheet
sheetNames = gSheet_utils.getSheetNames(spreadsheetID)

for sheetName in sheetNames:
    #Get name of org and parent org
    orgName = ' '.join(sheetName.split('memberships_')[1].split('_'))
    parentOrg = orgName.split('-')[0]
    
    #Import GSheet as df
    df, col_AI_map = gSheet_utils.importGSheetAsDF(sheetID, sheetName)
    gSheet_details = {'sheetID':sheetID, 'sheetName': sheetName, 'col_AI_map': col_AI_map, 'sub_langs':sub_langs}
    
    #If it's a sub org, import once under the parent org
    if parentOrg!=orgName:     
        orgID = searchCLI.searchCLI(base_url, parentOrg, 'organizations', 'name', 'othernames', []) #Find orgID from Popit
        print("Importing for {}".format(parentOrg)            
        df.apply(lambda row: membershipsUpdate_langs.genPayload(base_url, headers, row, orgID, gSheet_details, sub_langs), axis=1)

    #Import under the sub org
    orgID = searchCLI.searchCLI(base_url, orgName, 'organizations', 'name', 'othernames', [])
    print("Importing for {}".format(orgName))            
    df.apply(lambda row: membershipsUpdate_langs.genPayload(base_url, headers, row, orgID, gSheet_details, sub_langs), axis=1)


